## Data가 기존에 1.3GB => 400MB 까지 축소 가능

* [pandas handling](https://www.dataquest.io/blog/pandas-big-data/)

* Adversarial AutoEncoder 적용!!
* target에 따라서 latent space 를 잘 학습시킬수 있게하기.
* 시각화 하는 부분이나 2차원 맵핑해서 보여주는 것 만들어야 함.
* zdim 이 2차원으로 하다보니 잘 안된다 그래서 20차원으로 줄이고 또 이것을 2차원으로 줄인다면?...
    * 이게 사실 의미 있는 행위 인지는 모르겠다.

In [ ]:
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')


import numpy as np , os , pandas as pd , pandas_profiling
import missingno as msno
import matplotlib.pyplot as plt
import pprint , seaborn as sns

import tensorflow as tf
import seaborn as sns

from pprint import pprint

from moving_free_batch_normalization import moving_free_batch_norm
from stochastic_weight_averaging import StochasticWeightAveraging

from math import sin,cos,sqrt



In [ ]:
data = pd.read_csv("./credit44_sc.csv" )

In [ ]:
fac_var = []
num_var = []
for i in list(data) : 
    if data[i].nunique() < 15 : 
        print("factor variable : {}, count : {} ".format(i, data[i].nunique()))
        fac_var.append(i)
        if data[i].nunique() == 2 :
            print("binary" , i)
    else : 
        print("연속형 variable : {} , count : {} ".format(i, data[i].nunique()))
        num_var.append(i)
        
data = data.loc[:,fac_var + num_var]

col = list(data)

In [ ]:
fac_var = list(set(fac_var) - set(["target"]))

In [ ]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [ ]:
mem_usage(data)

In [ ]:
for dtype in ['float','int','object']:
    selected_dtype = data.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

In [ ]:
# We're going to be calculating memory usage a lot,
# so we'll create a function to save us some time!

data_int = data.select_dtypes(include=['int'])
converted_int = data_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(data_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([data_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

In [ ]:
data_float = data.select_dtypes(include=['float'])
converted_float = data_float.apply(pd.to_numeric,downcast='float')

print(mem_usage(data_float))
print(mem_usage(converted_float))

compare_floats = pd.concat([data_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

In [ ]:
gl_obj = data.select_dtypes(include=['object']).copy()

converted_obj = pd.DataFrame()

for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = gl_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = gl_obj[col]
        
        
print(mem_usage(gl_obj))
print(mem_usage(converted_obj))

compare_obj = pd.concat([gl_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

In [ ]:
#optimized_gl = data.copy()

data[converted_int.columns] = converted_int
data[converted_float.columns] = converted_float
data[converted_obj.columns] = converted_obj

print("Change Memory size : {}".format(mem_usage(data)))

```
msno.heatmap( data , figsize=(20,20))
plt.savefig("./IITP_EDA.png")
msno.dendrogram(data , figsize = (15,15))
plt.savefig("./IITP_EDA_Dandro.png")
msno.matrix(data)
plt.savefig("./IITP_MISSING_MATRIX.png")
plt.show()
```

In [ ]:


slim = tf.contrib.slim

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [ ]:
data_label = data.target
data = data.drop("target" , axis = 1 )


In [ ]:
data_label = pd.get_dummies(data_label , drop_first= False)

In [ ]:
data_label = data_label.values

In [ ]:
col =list(data)

In [ ]:
batch_size = 1000
z_id_ = np.random.randint(0, 2, size=[batch_size])

def gaussian_mixture(batch_size, n_dim=2, n_labels=2, x_var=0.5, y_var=0.1, label_indices=None):
    if n_dim != 2:
        raise Exception("n_dim must be 2.")

    def sample(x, y, label, n_labels):
        shift = 1.4
        r = 2.0 * np.pi / float(n_labels) * float(label)
        new_x = x * cos(r) - y * sin(r)
        new_y = x * sin(r) + y * cos(r)
        new_x += shift * cos(r)
        new_y += shift * sin(r)
        return np.array([new_x, new_y]).reshape((2,))

    x = np.random.normal(0, x_var, (batch_size, (int)(n_dim/2)))
    y = np.random.normal(0, y_var, (batch_size, (int)(n_dim/2)))
    z = np.empty((batch_size, n_dim), dtype=np.float32)
    for batch in range(batch_size):
        for zi in range((int)(n_dim/2)):
            if label_indices is not None:
                z[batch, zi*2:zi*2+2] = sample(x[batch, zi], y[batch, zi], label_indices[batch], n_labels)
            else:
                z[batch, zi*2:zi*2+2] = sample(x[batch, zi], y[batch, zi], np.random.randint(0, n_labels), n_labels)

    return z

In [ ]:
value = gaussian_mixture(batch_size , n_labels = 2 , label_indices= z_id_)
plt.scatter(value[:, 0], value[:, 1], c = z_id_ , s=40, cmap='viridis');

In [ ]:
def uniform(batch_size, n_dim, n_labels=10, minv=-1, maxv=1, label_indices=None):
    if label_indices is not None:

        def sample(label, n_labels):
            num = int(np.ceil(np.sqrt(n_labels)))
            size = (maxv-minv)*1.0/num
            x, y = np.random.uniform(-size/2, size/2, (2,))
            i = label / num
            j = label % num
            x += j*size+minv+0.5*size
            #y += i*size+minv+0.5*size
            return np.array([x, y]).reshape((2,))

        z = np.empty((batch_size, n_dim), dtype=np.float32)
        for batch in range(batch_size):
            for zi in range((int)(n_dim/2)):
                    z[batch, zi*2:zi*2+2] = sample(label_indices[batch], n_labels)
    else:
        z = np.random.uniform(minv, maxv, (batch_size, n_dim)).astype(np.float32)
    return z

In [ ]:
value = uniform(batch_size , n_dim =2 ,  n_labels = 2 , label_indices= z_id_)
plt.scatter(value[:, 0], value[:, 1], c = z_id_ , s=40, cmap='viridis');

In [ ]:


z_id_one_hot_vector = np.zeros((batch_size, 2))
z_id_one_hot_vector[np.arange(batch_size), z_id_] = 1

In [ ]:
mb_size = 1500
X_dim = np.shape(data)[1]
label_n = data_label.shape[1]
z_dim = 2
lr = 0.0001
beta_1 = 0.5
beta_2 = 0.9
d_steps = 5
g_steps = 3
EPOCHS = 50000
Margin = 14
Margin_LIMIT = Margin - 1 
ROW , COL = 7,7
data_len = np.shape(data)[0]
batch_iter = int(data_len / mb_size)
LAMDA = 10 
Recon_weight = 2

seed_n = np.random.randint(1, 1000, size=1 )[0]

tf.random.set_random_seed(seed_n)


remark = "SAMPLE_Z를 Uniform 에서 Gaussian으로 변경? 근데 Uniform으로 하면 더 차이를 볼 수 있지 않을까?"
title = "0310_AAE_v5"

path_v = title

loss_func = "rasgan_gp"

model_dir = "./Model_Save/{}".format(path_v)
vis_dir =  model_dir + "/vis"

if not os.path.exists(model_dir) :
    os.makedirs(model_dir)
    os.makedirs(vis_dir)

if loss_func == "ragan_gp" :
    Gradient_type = ["GP","LP"][np.random.randint(0, 2, size=1 )[0]]
else :
    Gradient_type = "None" 
    
DATA_RANGE = (0,1)


select1 = np.random.randint(0, 2, size=1 )[0]
select2 = np.random.randint(0, 2, size=1 )[0]

relu_w_init = [tf.keras.initializers.he_uniform(seed = seed_n) ,
               tf.keras.initializers.he_normal(seed = seed_n)][select1]
tanh_w_init = [tf.keras.initializers.glorot_normal(seed = seed_n) ,
               tf.keras.initializers.glorot_uniform(seed = seed_n)][select2]



relu_w_say = ["he uniform" , "he normal"][select1]


if DATA_RANGE == (0,1) :
    g_act = tf.nn.sigmoid
    tanh_w_say = "None"
if DATA_RANGE == (-1,1) :
    g_act = tf.nn.tanh
    tanh_w_say = ["glorot normal" , "glorot uniform"][select2]


spec_iter = np.random.randint(1,4, size = 1)[0]

P_layer_count = np.random.randint(5, 7, size=1 )[0]
Q_layer_count = np.random.randint(5, 7, size=1 )[0]
D_layer_count = P_layer_count 

P_layer = list(np.sort(np.random.randint(5 , z_dim + label_n + 10, size=P_layer_count ))[::-1]) ## Discriminator
D_layer = list(np.sort(np.random.randint(z_dim + 2 , X_dim - 2 , size=D_layer_count ))) ## Decoder
Q_layer = list(np.sort(np.random.randint(z_dim + 2 , 55 , size=Q_layer_count ))[::-1]) ## Encoder
    
P_H = [ z_dim + label_n ] + P_layer + [ 1 ]
D_H = [ z_dim ] + D_layer + [ X_dim ]
Q_H = [ X_dim ] + Q_layer + [ z_dim ]




tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[None, X_dim], name="X")
Z = tf.placeholder(tf.float32, shape=[None, z_dim], name="z")
Z_ID = tf.placeholder(tf.float32, shape=[None, label_n], name='prior_sample_label')
X_ID = tf.placeholder(tf.float32, shape=[None, label_n], name='target_label')
Z_IN = tf.placeholder(tf.float32, shape=[None, z_dim], name='latent_variable')

batch_size = tf.placeholder(tf.int64)
dataset = tf.data.Dataset.from_tensor_slices((X, Z , Z_ID , X_ID, Z_IN )).batch(batch_size ,drop_remainder= True ).repeat().shuffle(1000)

#is_training_bn = tf.placeholder(shape=[], dtype=tf.bool)
#use_moving_statistics = tf.placeholder(shape=[], dtype=tf.bool)
learning_rate = tf.placeholder(shape=[], dtype=tf.float32)


iter = dataset.make_initializable_iterator()
x , z , z_id , x_id , z_in = iter.get_next()
print("="*30)
print("Title : {}".format(title))
print("Loss func : {}".format(loss_func))
print("Model Directory : {}".format(model_dir))
print("Gradient Type : {} , Lambda : {}".format(Gradient_type , LAMDA ))
print("Data RANGE : {} , G activation : {}".format(DATA_RANGE , g_act))
print("Q Hidden : {}".format(P_H))
print("D Hidden : {}".format(D_H))
print("H Hidden : {}".format(Q_H))
print("Dim : {} , Z dim : {}, Margin : {}".format(np.shape(data) , z_dim , Margin))
print("Epoch : {} , Batch SIZE : {} , Batch Iter : {}".format(EPOCHS , mb_size , batch_iter))
print(" Beta1 : {} , Beta2 : {}".format( beta_1 , beta_2))
print("D Step : {} , G Step : {}".format(d_steps , g_steps))
print("="*30)

inner = [[remark , title , loss_func , model_dir , Gradient_type , LAMDA , spec_iter , 
          Recon_weight , seed_n , DATA_RANGE , g_act , P_H  , D_H , relu_w_say , tanh_w_say , 
 np.shape(data) , z_dim , Margin , EPOCHS , mb_size , batch_iter , beta_1 , beta_2, d_steps , g_steps]]

colname = ["Remark : " ,  "Title : " , "Loss function : " , "Model 경로 : " , "Gradient 타입 : " , "LAMDA : " ,  "spec iter" , 
           "Reconstrunction weight : " ,  "seed" ,
           'G 생성 범위 : ' , 'G 생성 함수 : ' ,"G Hidden SIZE : " , "D Hidden SIZE : " , 
           "Relu W init" , "Tanh W init" , 
           "Data Shape : " , " Z dim : " ,
           "Margin : ", "Epochs : " , "Batch size  : " , 
           " Batch Iteration : " , "Beta 1 : " , "Beta 2 : " ,
           "D steps : " , "G steps  : " ]

pd.DataFrame( inner , columns = colname).T.to_csv(model_dir + "/summary.csv")

In [ ]:
def sample_z(m, n_dim , n_labels=2, minv=-1, maxv=1, label_indices= None ):
    if label_indices is not None:

        def sample(label, n_labels):
            num = int(np.ceil(np.sqrt(n_labels)))
            size = (maxv-minv)*1.0/num
            x, y = np.random.uniform(-size/2, size/2, (2,))
            i = label / num
            j = label % num
            x += j*size+minv+0.5*size
            #y += i*size+minv+0.5*size
            return np.array([x, y]).reshape((2,))

        z = np.empty((m, n_dim), dtype=np.float32)
        for batch in range(m):
            for zi in range((int)(n_dim/2)):
                    z[batch, zi*2:zi*2+2] = sample(label_indices[batch], n_labels)
    else:
        z = np.random.uniform(minv, maxv, (m, n_dim)).astype(np.float32)
    return z


def sample_z(batch_size, n_dim=2, n_labels=2, x_var=0.5, y_var=0.1, label_indices=None):

    def sample(x, y, label, n_labels):
        shift = 1.4
        r = 2.0 * np.pi / float(n_labels) * float(label)
        new_x = x * cos(r) - y * sin(r)
        new_y = x * sin(r) + y * cos(r)
        new_x += shift * cos(r)
        new_y += shift * sin(r)
        return np.array([new_x, new_y]).reshape((2,))

    x = np.random.normal(0, x_var, (batch_size, (int)(n_dim/2)))
    y = np.random.normal(0, y_var, (batch_size, (int)(n_dim/2)))
    z = np.empty((batch_size, n_dim), dtype=np.float32)
    for batch in range(batch_size):
        for zi in range((int)(n_dim/2)):
            if label_indices is not None:
                z[batch, zi*2:zi*2+2] = sample(x[batch, zi], y[batch, zi], label_indices[batch], n_labels)
            else:
                z[batch, zi*2:zi*2+2] = sample(x[batch, zi], y[batch, zi], np.random.randint(0, n_labels), n_labels)

    return z

In [ ]:
def layer(prev = None , shape1 = None , shape2 = None , 
          name = None , activation = tf.nn.leaky_relu ,
          final = False , SN = True , Type = None) :
    
    if activation == tf.nn.leaky_relu or activation == tf.nn.relu : 
        init = relu_w_init
    
    elif activation == tf.nn.tanh :
        init = tanh_w_init
    
    W1 = tf.get_variable("Weight" + str(name) , shape = [shape1 , shape2] , dtype = tf.float32 ,
                        initializer = init)
    b1 = tf.get_variable("Bias" + str(name) , shape = [shape2] , dtype = tf.float32 ,
                        initializer = tf.constant_initializer(0.001))
    W2 = spectral_norm(W1 , name = "SN" + str(name))
    if final == True :
        layer = tf.matmul( prev , W1 )
        layer = activation(layer)
    else :
        if SN == True :
            layer = tf.matmul( prev , W2) + b1
            layer = activation(layer)
        else : 
            layer = tf.matmul( prev , W1) + b1
            if Type == "SWA" :
                layer = moving_free_batch_norm(layer, axis=-1, training=is_training_bn,
                                               use_moving_statistics=use_moving_statistics, momentum=0.99)
                layer = activation(layer)
            elif Type == "Self_Normal" :
                layer = activation(layer)
                layer = tf.contrib.nn.alpha_dropout(layer , 0.5)
            elif Type == "Instance_Norm" :
                layer = tf.contrib.layers.instance_norm(layer)
                layer = activation(layer)
            else :
                ## Nothing
                layer = layer
    return layer


"""
MLP ENCODER
"""     
def Q(X = None , reuse = tf.AUTO_REUSE) :
    
    with tf.variable_scope("Q" , reuse = reuse) as scope: 
        h = layer( prev = X , shape1 = Q_H[0], shape2 = Q_H[1] ,
                  name = "Q_first", SN = False ,Type = "N" )
        for i in range( len(Q_H) ) :
            if (i + 2) >= (len(Q_H) -1 ) :
                continue
            else :
                h = layer(prev= h , shape1 = Q_H[i+1], shape2 = Q_H[i+2] ,
                          name = "Q" + str(i) , SN = False ,Type = "N")
        Out = layer(prev = h , shape1 = Q_H[-2] , shape2 = Q_H[-1] ,
                   name = "Encoder_final", final = True)
        
        return Out


"""
Discriminator
"""  

def P(Z = None , reuse = tf.AUTO_REUSE) :
    with tf.variable_scope("P" , reuse = reuse) as scope: 
        h = layer(prev = Z , shape1 = P_H[0], shape2 = P_H[1] ,
                  name = "P_first", SN = True ,  Type = "N")
        for i in range( len(P_H) ) :
            if (i + 2) >= (len(P_H) -1 ) :
                continue
            else :
                h = layer(prev= h , shape1 = P_H[i+1], shape2 = P_H[i+2] ,
                          name = "P" + str(i) , SN = True, Type = "N")
        Out = layer(prev = h , shape1 = P_H[-2] , shape2 = P_H[-1] ,
                   name = "W_P_final", final = True)
        
        if g_act == tf.nn.sigmoid :
            G_final_b = tf.get_variable("Discriminator_Bias_final" , shape = [P_H[-1]],
                                       dtype = tf.float32 , initializer = tf.constant_initializer(0.001))
            Out  = Out + G_final_b
            prob = g_act(Out)
        else :
            prob = g_act( Out )
        return prob , Out 
      
"""
MLP DECODER 나중에 확률 값 내뱉기  (0~1로 만들기)
"""      
def D(Z = None , reuse = tf.AUTO_REUSE) :
    with tf.variable_scope("D" , reuse = reuse) as scope: 
        h = layer(prev = Z , shape1 = D_H[0], shape2 = D_H[1] ,
                  name = "D_first", SN = False ,Type = "N")
        for i in range( len(D_H) ) :
            if (i + 2) >= (len(D_H) -1 ) :
                continue
            else :
                h = layer(prev= h , shape1 = D_H[i+1], shape2 = D_H[i+2] ,
                          name = "D" + str(i) , SN = False, Type = "N")
        Out = layer(prev = h , shape1 = D_H[-2] , shape2 = D_H[-1] ,
                   name = "W_D_final", final = True)
        
        if g_act == tf.nn.sigmoid :
            G_final_b = tf.get_variable("Decoder_Bias_final" , shape = [D_H[-1]],
                                       dtype = tf.float32 , initializer = tf.constant_initializer(0.001))
            Out  = Out 
            #+ G_final_b
            prob = g_act(Out)
        else :
            prob = g_act( Out )
            
        return prob

    
def Discriminator_loss( Ra = None, loss_func = None, real = None, fake = None) :
    loss = 0
    if Ra :
        fake_logit = (fake - tf.reduce_mean(real))
        real_logit = (real - tf.reduce_mean(fake))
        
        if loss_func == "ralsgan" :
            fake_loss = tf.reduce_mean( tf.square( fake_logit + 1 ))
            real_loss = tf.reduce_mean( tf.square( real_logit - 1 ))
            loss = real_loss + fake_loss
        if loss_func == "rasgan_gp" or loss_func == "rasgan" :
            real_logit = tf.sigmoid( real_logit )
            fake_logit = tf.sigmoid( fake_logit )
            loss = - tf.reduce_mean( log(real_logit)) - tf.reduce_mean( log(1-fake_logit ))
        if loss_func == "rsgan_gp" or loss_func == "rsgan" :
            loss = - tf.reduce_mean( log(tf.nn.sigmoid( real - fake)))
        if loss_func == "rahinge" :
            fake_loss = tf.reduce_mean( tf.nn.relu( 1 + fake_logit ))
            real_loss = tf.reduce_mean( tf.nn.relu( 1 - real_logit ))
            loss = real_loss + fake_loss
        if loss_func == "Boundary" : 
            #real_logit = tf.sigmoid( real - fake)
            #fake_logit = tf.sigmoid( fake - real )
            real_logit = tf.sigmoid( real_logit )
            fake_logit = tf.sigmoid( fake_logit )
            loss  =  -tf.reduce_mean(log( real_logit ) + log( 1 - fake_logit ))
    
    return loss

def Generator_loss(Ra  = None , loss_func  = None , real = None ,fake = None ) :
    loss = 0
    if Ra :
        fake_logit = (fake - tf.reduce_mean(real))
        real_logit = (real - tf.reduce_mean(fake))
        
        if loss_func == "ralsgan" :
            fake_loss = tf.reduce_mean( tf.square( fake_logit - 1 ))
            real_loss = tf.reduce_mean( tf.square( real_logit + 1 ))
            loss = real_loss + fake_loss
        if loss_func == "rasgan_gp" or loss_func == "rasgan" :
            real_logit = tf.sigmoid( real_logit )
            fake_logit = tf.sigmoid( fake_logit )
            loss = - tf.reduce_mean( log( fake_logit )) - tf.reduce_mean( log(1- real_logit ))
        if loss_func == "rsgan_gp" or loss_func == "rsgan" :
            loss = - tf.reduce_mean( log(tf.nn.sigmoid( fake - real)))
        if loss_func == "rahinge" :
            fake_loss = tf.reduce_mean( tf.nn.relu( 1 - fake_logit ))
            real_loss = tf.reduce_mean( tf.nn.relu( 1 + real_logit ))
            loss = real_loss + fake_loss
        if loss_func == "Boundary" : 
            fake_logit = tf.sigmoid( fake_logit )
            #fake_logit = tf.sigmoid( fake - real  )
            loss  =  0.5 * tf.reduce_mean( ( log( fake_logit ) - log( 1 - fake_logit ) )**2 )
    return loss

    

# def sample_z(m, n) :
#     output = np.random.normal(loc = 0 ,scale= 1 , size=[m, n])
#     #output = np.random.uniform(-1., 1., size=[m, n])
#     return output

def log(x):
    return tf.log( tf.maximum( x , 1e-8) )



def spectral_norm(w, iteration= spec_iter , name = None):
    w_shape = w.shape.as_list()
    w = tf.reshape(w, [-1, w_shape[-1]])
    
    u = tf.get_variable(name , [1, w_shape[-1]], initializer=tf.random_normal_initializer(), trainable=False)

    u_hat = u
    v_hat = None
    for i in range(iteration):
       
        """
       power iteration
       Usually iteration = 1 will be enough
       """
        
        v_ = tf.matmul(u_hat, tf.transpose(w))
        v_hat = tf.nn.l2_normalize(v_)

        u_ = tf.matmul(v_hat, w)
        u_hat = tf.nn.l2_normalize(u_)

    u_hat = tf.stop_gradient(u_hat)
    v_hat = tf.stop_gradient(v_hat)

    sigma = tf.matmul(tf.matmul(v_hat, w), tf.transpose(u_hat))

    with tf.control_dependencies([u.assign(u_hat)]):
        w_norm = w / sigma
        w_norm = tf.reshape(w_norm, w_shape)

    return w_norm 




In [ ]:
z_sample                  = Q(x) ## ENCODER
X_samples                 = D(z_sample) ## DECODER

z_real = tf.concat([z, z_id],1)
z_fake = tf.concat([z_sample, x_id],1)
D_fake, D_fake_logits     = P(z_fake) ## DISCRIMI
D_real, D_real_logits     = P(z_real)


Test_Samples = D( z_in )
"""
z : encoder 값 -> 이것이 fake 가 되야 함
z_sample :  임의의 분포 값 f

"""

recon_loss = tf.reduce_mean(tf.reduce_sum(tf.squared_difference(x , X_samples ) , axis = 1))




#D_loss = -tf.reduce_mean( log(D_real) +  log(1. - D_fake))
#G_loss = -tf.reduce_mean( log(D_fake) ) 

e = tf.random_uniform([batch_size , 1] , 0 , 1)
x_hat = e * z_real + (1-e) * z_fake
grad  = tf.gradients( P(x_hat), x_hat)[0]
grad_norm = tf.norm( tf.layers.flatten(grad) , axis = 1)

if Gradient_type == "GP" :
    GP = LAMDA * tf.reduce_mean( tf.square( grad_norm - 1.0))
elif Gradient_type == "LP" :    
    GP = LAMDA * tf.reduce_mean( tf.square( tf.maximum(0.0 ,grad_norm - 1.0)))
else :
    GP = 0
    

D_loss = Discriminator_loss(True , "Boundary" , D_real_logits , D_fake_logits) + GP

# D_loss_real = tf.reduce_mean(
#         tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real_logits)))
# D_loss_fake = tf.reduce_mean(
#     tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros_like(D_fake_logits)))
# D_loss = D_loss_real+D_loss_fake + GP

    
G_loss = Generator_loss(True , "Boundary" , D_real_logits , D_fake_logits)

## 함수가 점점 감소하는 것은 볼 수 있지만 결과가 좋아지는 것을 볼 수가 없다. 

# G_loss = tf.reduce_mean(
#     tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits,
#                                             labels=tf.ones_like(D_fake_logits)))


In [ ]:


# print(GP)
# D_loss = Discriminator_loss(True , loss_func , D_real , D_fake) + GP
# G_loss = Generator_loss(True , loss_func , D_real , D_fake)

# tf.contrib.framework.sort()
#gene_17 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_sample), 17))
#true_17 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(x), 17))
# gene_17 = gene_17 / tf.reduce_sum(gene_17)
# true_17 = true_17 / tf.reduce_sum(true_17)


# def kl_divergence(p, q): 
#     return tf.reduce_sum(p * log( p/q  ))



# recon_loss_17 = kl_divergence(true_17 , gene_17 )

# recon_loss = Recon_weight * recon_loss_17


In [ ]:
t_vars = tf.trainable_variables()

d_vars = [var for var in t_vars if "P" in var.name]
g_vars = [var for var in t_vars if "D" in var.name]
recon_vars = [var for var in t_vars if "D" or "Q" in var.name]
#pp.pprint(g_vars)
#print("\n")
#pp.pprint(d_vars)

In [ ]:
#update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
#update_bn_ops = tf.get_collection('UPDATE_BN_OPS')
#reset_bn_ops = tf.get_collection('RESET_BN_OPS')
#update_ops = tf.group(*update_ops)
#update_bn_ops = tf.group(*update_bn_ops)
#reset_bn_ops = tf.group(*reset_bn_ops)



In [ ]:
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    G_solver = tf.train.AdamOptimizer(learning_rate=learning_rate , beta1=beta_1 , beta2 = beta_2).minimize(G_loss, var_list=g_vars)
    D_solver = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta_1 , beta2 = beta_2).minimize(D_loss, var_list=d_vars)
    AE_solver = tf.train.AdamOptimizer().minimize(recon_loss, var_list=recon_vars )
    


In [ ]:
# with tf.name_scope('SWA'):
#     swa = StochasticWeightAveraging()
#     swa_op = swa.apply(var_list=g_vars)
#     # Make backup variables
#     with tf.variable_scope('BackupVariables'):
#         backup_vars = [tf.get_variable(var.op.name, dtype=var.value().dtype, trainable=False,
#                                        initializer=var.initialized_value())
#                        for var in g_vars ]
#     # operation to assign SWA weights to model
#     swa_to_weights = tf.group(*(tf.assign(var, swa.average(var).read_value()) for var in g_vars))
#     # operation to store model into backup variables
#     save_weight_backups = tf.group(*(tf.assign(bck, var.read_value()) for var, bck in zip(g_vars, backup_vars)))
#     # operation to get back values from backup variables to model
#     restore_weight_backups = tf.group(*(tf.assign(var, bck.read_value()) for var, bck in zip(g_vars, backup_vars)))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def sample_data(data = None , n= len(data)) : 
    output = data[list(np.random.choice(len(data) , n))]
    
    return output

scaler = MinMaxScaler(feature_range= DATA_RANGE )

x_plot = pd.DataFrame(sample_data(data = data.values , n=mb_size) , columns = col)
data = scaler.fit_transform(data)

In [ ]:
def Check_KS(ks_output , Total_ks_graph , Margin , ROW , COL ) : 
    clear_output(wait= True)
    print("KS Plot")
    total = sess.run(Test_Samples )
    total = scaler.inverse_transform(total)
    g_plot = pd.DataFrame(total , columns = col )
    g_plot[fac_var] = g_plot[fac_var].round(0)
    Value = []
    for label in col :
        sample = g_plot[label]
        real   = x_plot[label]
        ks , p = stats.ks_2samp(real.values , sample.values)
        Value.append(ks)
    Total_KS = round( np.sum(Value) , 2)
    ks_2 = [iteration] + Value
    ks_3 = pd.DataFrame([ks_2], columns = ["iter"] + col)
    ks_output = ks_output.append(ks_3)
    fig , ax = plt.subplots(figsize=(26,13))
    fig.subplots_adjust(top = 0.95 , left = 0.03 , bottom = 0.05 , right = 0.99)
    updown = 0
    for name in col : 
        if updown % 2 == 0 :
            param , space="bottom" , "  "
        else : 
            param , space ="top" , "   "
        ax.plot(ks_output.iter , ks_output[[name]], label = name)
        ax.text(iteration , ks_output.loc[ks_output["iter"]==iteration , [name]].values , space + name ,
                verticalalignment = param)
        updown +=1
    ax.set_title("KS [{}]".format(Total_KS) , fontsize = 30 )
    ax.set_xlabel("Epoch")
    ax.set_ylabel("KS")    
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9])
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=15 , fontsize= 10)
    ax.text(iteration+1 , 0.05 , "   0.05", verticalalignment = param)
    ax.axhline(0.05, linewidth=4, color='r')
    ax.set_title("[{}] , EPOCH : {} , KS : {}[{}][{}]".format(title , iteration-1 , Total_KS , Total_ks_graph.ks.min() ,len(col) ) , fontsize = 30)
    plt.savefig(model_dir +"/IND_KS_Log_{}.png".format(path_v))
    plt.show()
    """
    Total Graph
    """
    
    print("Total KS Plot")
    Total_ks_graph_ap = pd.DataFrame({"iter":[iteration] , "ks" :[Total_KS]})
    Total_ks_graph    = Total_ks_graph.append(Total_ks_graph_ap)
    #Total_ks_graph    = Total_ks_graph[Total_ks_graph.iter>0]
    fig , ax = plt.subplots(figsize = (26,13))
    fig.subplots_adjust(top = 0.95 , left = 0.03 , bottom = 0.04 , right = 0.99)
    ax.plot(Total_ks_graph.iter , Total_ks_graph.ks , linestyle ="-" , marker ="." , linewidth = 3, markersize = 12)
    ax.axhline(0.05, linewidth=4, color='r')
    ax.set_title("[{}] , EPOCH : {} , KS : {}[{}][{}]".format(title , iteration-1 , Total_KS , Total_ks_graph.ks.min() ,len(col) ) , fontsize = 30)
    plt.savefig(model_dir +"/Total_KS_Log{}.png".format(path_v))
    plt.show()
    
    fig , ax = plt.subplots(figsize = (26,13))
    fig.subplots_adjust(top = 0.95 , left = 0.03 , bottom = 0.04 , right = 0.99)

    ax.plot(output.iter , output.dloss , label ="dloss" , linestyle ="-" , marker ="." , linewidth = 4, markersize = 12)
    ax.plot(output.iter , output.gloss , label ="gloss" , linestyle ="-" , marker ="." , linewidth = 4, markersize = 12)
    ax.set_title("[{}] , EPOCH : {} , Dloss : {} , Gloss : {}".format(title , iteration-1 ,  dloss, gloss ), fontsize= 30)
    #ax.set_ylim(-5, 15)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9])
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=4 , fontsize= 20)
    plt.savefig(model_dir +"/LOSS_Log_{}.png".format(path_v))
    plt.show()
    
    
    if Margin > Total_KS : 
        show_plot(row = ROW , ncol= COL )
        print("=======UPDate============")
        with open(model_dir + "/Change_Margin.txt", "a") as f:
            f.write("Epoch : {} , Margin : {} ===> {} \n".format(iteration , Margin , Total_KS))
        Margin = Total_KS
        ## 여기선 특정 샘플 저장해야 하므로. 원하는 개수 만큼.
        generate = {}
        sample_n = 50000
        z_id_ = np.random.randint(0, 2, size=[sample_n])
        z_id_one_hot_vector = np.zeros((sample_n, 2))
        z_id_one_hot_vector[np.arange(sample_n), z_id_] = 1
        z_batch = sample_z(sample_n , z_dim , label_indices= z_id_)
        generate[z_in] = z_batch
        total = sess.run(Test_Samples, feed_dict= generate )
        total = scaler.inverse_transform(total)
        g_plot = pd.DataFrame(total , columns = col )
        g_plot[fac_var] = g_plot[fac_var].round(0)
        g_plot.to_csv(model_dir + "/Generated_{}.csv".format(path_v) , index = False)
        
        saver.save(sess , model_dir + "/MODEL_{}".format(path_v))
    return ks_output , Total_ks_graph , Margin , Total_KS

In [ ]:
col

In [ ]:
from scipy import stats
def show_plot(row , ncol ) : 
    
    fig , axes = plt.subplots(row , ncol , figsize = (26,13))
    fig.subplots_adjust(hspace = 0.35 , wspace= 0.14 , top = 0.92 , left = 0.03 , bottom = 0.04 , right = 0.99)
    total = sess.run( Test_Samples )
    try : 
        total = total[~np.isnan(total).any(axis=1)]
        total = scaler.inverse_transform(total)
        g_plot = pd.DataFrame(total , columns = col )
        g_plot[fac_var] = g_plot[fac_var].round(0)
        """
        좀 더 쉬운 분포로 만들어서 학습시킨 후 다시 원래값으로 (factor 변수이기 때문에 가능하다 생각함.)
        """

        col2 = 0
        error = []
        for j in range(row) :
            for k in range(ncol) :
                try :
                    label = col[col2]
                    sample = g_plot.loc[: , label]
                    sample.name = "Gene"
                    real_0 = x_plot.loc[: , label]
                    real_0.name ="Real"
                    ks , p = stats.ks_2samp(real_0.values , sample.values)
                    error.append(ks)
                    col2 += 1
                    if label in fac_var : 
                        sns.distplot( sample , ax=axes[j , k], norm_hist =True , kde=False , hist_kws ={"color":"r" , "label" :"Gene", "rwidth":0.75})
                        sns.distplot(real_0 , ax=axes[j , k],norm_hist =True, kde=False , hist_kws ={"color":"g" , "label" :"Real", "rwidth":0.75})
                        axes[j , k].legend(fontsize = 10)
                    elif label in num_var : 
                        sns.distplot(  sample , ax=axes[j , k] ,
                                     kde_kws={"color": "r", "lw": 2, "label": "Gene" , "shade" : True} , hist =False , rug = False) #   
                        sns.distplot(  real_0 , ax=axes[j , k] ,
                                     kde_kws={"color": "g", "lw": 2, "label": "Real", "shade" : True } , hist =False , rug = False) # 
                        axes[j , k].legend(fontsize = 10 )
                    axes[j , k].set_title( label , loc ="center" , fontsize= 10 )
                    axes[j , k].set_xlabel(' ')
                except IndexError as e : 
                    axes[j , k].axis("off")
        
        KS_DIF = round(np.sum(error),2)
        plt.suptitle('[{}] EPOCH {} , D_loss : {} , G_loss : {} KS : {}'.format(title , i , dloss,gloss,KS_DIF) , fontsize= 30)
        if KS_DIF < 9 : 
            plt.savefig(vis_dir +"/Vis_{}_{}.png".format(path_v,KS_DIF))
        else : 
            plt.savefig(model_dir +"/Vis_{}.png".format(path_v))
        plt.show()
        
        return print("시각화")

    except Exception as e : 
        print(e)

In [ ]:
EPOCHS_BEFORE_SWA = 500
ALPHA1_LR = 0.0009
ALPHA2_LR = 0.000001

def get_learning_rate(step, epoch, steps_per_epoch):
    if epoch < EPOCHS_BEFORE_SWA:
        return ALPHA1_LR

    if step > int(0.9 * EPOCHS * steps_per_epoch):
        return ALPHA2_LR

    length_slope = int(0.9 * EPOCHS * steps_per_epoch) - EPOCHS_BEFORE_SWA * steps_per_epoch
    return ALPHA1_LR - ((ALPHA1_LR - ALPHA2_LR) / length_slope) * (step - EPOCHS_BEFORE_SWA * steps_per_epoch)

steps_per_epoch_train = EPOCHS// mb_size
print(steps_per_epoch_train)

In [ ]:
all_steps = []
all_lr = []

step = 0
for epoch in range(EPOCHS):
    for _ in range(steps_per_epoch_train):
        all_steps.append(step)
        all_lr.append(get_learning_rate(step, epoch, steps_per_epoch_train))
        step += 1 

plt.figure(figsize=(16,4))
plt.plot(all_steps, all_lr)

In [ ]:
# from math import ceil

# steps_per_epoch_train = int(ceil(EPOCHS/mb_size))
# def fit_bn_statistics():
#     sess.run(reset_bn_ops)
    
#     feed_dict = {is_training_bn: True, use_moving_statistics: True}
#     for _ in range(steps_per_epoch_train):
#         sess.run(update_bn_ops, feed_dict=feed_dict)

# def inference(with_moving_statistics=True):
#     feed_dict = {is_training_bn: False,
#                  use_moving_statistics: with_moving_statistics}
#     g_loss = []
#     d_loss = []
#     nb_steps = 10
    
#     for _ in range(nb_steps):
#         _, Dloss = sess.run([D_solver, D_loss], feed_dict=feed_dict_train)
#         _, Gloss = sess.run([G_solver, G_loss], feed_dict=feed_dict_train)
#         g_loss.append(Gloss)
#         d_loss.append(Dloss)
    
#     return np.mean(g_loss), np.mean(d_loss)

# feed_dict_train = {is_training_bn: True, 
#                    use_moving_statistics:True,}

feed_dict_train = {}

In [ ]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

In [ ]:

print("START")
config=tf.ConfigProto( log_device_placement=True)
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()


ks_output = pd.DataFrame([[0]+[1.]*len(col)], columns = ["iter"]+col)

iteration , dloss ,  gloss = 0 , 0, 0
output = pd.DataFrame({"iter" : [iteration] , "dloss" : [dloss] , "gloss" : [gloss] })
ks_init = len(col)
Total_ks_graph = pd.DataFrame({"iter":[iteration] , "ks" :[ks_init]})

from MulticoreTSNE import MulticoreTSNE as TSNE


step = 0

for i in range(EPOCHS) : 
    z_id_ = np.random.randint(0, 2, size=[data_len])
    z_id_one_hot_vector = np.zeros((data_len, 2))
    z_id_one_hot_vector[np.arange(data_len), z_id_] = 1
    z_batch = sample_z(data_len , z_dim , label_indices= z_id_)
    sess.run(iter.initializer, feed_dict={ X : data , Z : z_batch  ,
                                          batch_size: mb_size ,
                                          Z_ID : z_id_one_hot_vector, X_ID : data_label , Z_IN : z_batch })
    iteration +=1
    dloss = []
    gloss = []
    for _ in range( batch_iter)  :
        feed_dict_train[learning_rate] = get_learning_rate(step, i , steps_per_epoch_train)
        feed_dict_train[batch_size] = mb_size
        step += 1
        for _ in range(5) :
            _, Dloss = sess.run([D_solver, D_loss], feed_dict=feed_dict_train)
            dloss.append(Dloss)
        
        for _ in range(2) :
            _ , Gloss = sess.run([G_solver, G_loss], feed_dict=feed_dict_train)
            gloss.append(Gloss)
            _, recon = sess.run([AE_solver, recon_loss], feed_dict=feed_dict_train )
        
        
        
        
    dloss = np.mean(dloss)
    gloss = np.mean(gloss)
    
    
#     sess.run(swa_op)
#     sess.run(save_weight_backups)
#     sess.run(swa_to_weights)
#     sess.run(restore_weight_backups)
    
#     fit_bn_statistics()
#     dloss, gloss = inference(with_moving_statistics=False)
    
    output1 = pd.DataFrame({"iter" : [iteration] , "dloss" : [dloss] , "gloss" : [gloss] })
    output  = output.append(output1)
    output  = output[output.iter > 0 ]
    dloss = round( np.float64(dloss),4)
    gloss = round( np.float64(gloss),4)
    ks_output , Total_ks_graph , Margin  , Total_KS = Check_KS(ks_output , Total_ks_graph , Margin , ROW , COL)
    print('EPOCH : {}; D_loss: {}; G_loss: {} , KS : {}'.format(i, dloss, gloss , Total_KS))
    print("Recon Loss : {}".format(recon))
    
    
    
    import random
#     idx = np.random.choice(data_len, 1000 , replace=False )
#     sample_manifold_label = data_label[idx]
#     sample_manifold       = data[idx]
    z_PMLR = sess.run(z_sample , feed_dict = {x : data})
    if z_dim == 2 :
        z_PMLR = z_PMLR
    else :
        z_PMLR = TSNE(n_components=2, perplexity=15, learning_rate=0.01 , n_jobs=10).fit_transform(z_PMLR)
        
    N = 2 
    plt.figure(figsize=(10, 10))
    plt.scatter(z_PMLR[:, 0], z_PMLR[:, 1], c=np.argmax(data_label, 1),
                marker='o', edgecolor='none', cmap=discrete_cmap(N, 'jet'))
    plt.colorbar(ticks=range(N))
    axes = plt.gca()
    #axes.set_xlim([-1, 1])
    #axes.set_ylim([-1, 1])
    plt.grid(True)
    plt.savefig(vis_dir +"/Manifold.png")
    plt.show()

        
    if (i % 10 == 0) &  (Margin < Margin_LIMIT ) :
        clear_output(wait= True)
        show_plot(row = ROW , ncol= COL)

